In [2051]:
#we use set for taken and not taken, is better than vector beacause we don't have position
import numpy as np

In [2052]:
from random import random
import numpy as np
import operator
from functools import reduce
from queue import PriorityQueue
from collections import namedtuple
from math import ceil


In [2053]:
PROBLEM_SIZE=5
NUM_SETS=10
State=namedtuple("State",["taken","not_taken"])

In [2054]:
#one possible set could be:
SETS=tuple(np.array([random()<.3 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
SETS

(array([False, False, False,  True,  True]),
 array([ True, False,  True, False, False]),
 array([False, False,  True, False, False]),
 array([False, False, False, False, False]),
 array([ True, False,  True,  True, False]),
 array([False,  True,  True, False,  True]),
 array([ True, False, False, False,  True]),
 array([ True, False, False, False,  True]),
 array([ True, False,  True, False, False]),
 array([False,  True, False, False,  True]))

In [2055]:
state=({1,3,5},{0,2,4,6,7})#in the first position there are step taken and the second are not taken position
#how can i calculate the goal state?
[SETS[i] for i in state[0]]
np.all(reduce(np.logical_or,[SETS[i] for i in state[0]]))#our taste to for understand if we are in the goal state

False

In [2056]:
def goal_check(state):
    return np.all(reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)])))

# MY SOLUTION
What I tried to do to improve the Heuristic professor’s solution is to reorder the vector of not taken based on the number of true cells and extract the first N elements whose sum of true was greater than the true missing cells, N will be my result.

In [2057]:
def array_with_more_true(arrays, missing):
    num_true_per_array = [np.count_nonzero(array) for array in arrays]#create a vector with number of cell true 
    array_with_count = list(zip(arrays, num_true_per_array))#create an array with sets, n_true
    array_with_count.sort(key=lambda x: x[1], reverse=True)#reorder
    element_in_arrays = [x[1] for x in array_with_count]
    cnt=0#cnt si incrementa fino a che non copro tutte le missing size
    cov=missing
    for i in element_in_arrays:
       if missing>0:
           cnt+=1
           missing=missing-i
    top_n_arrays = [x[0] for x in array_with_count[:cnt]]
    print("mancano:",cov,"e possiamo usare nel caso migliore",cnt, "sets e sono:\n",top_n_arrays,"\n\n")

    return top_n_arrays,cnt

def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )

def h(state):
    
    missing_size = PROBLEM_SIZE - sum(covered(state))
    x=tuple(SETS[i] for i in state.not_taken)
    _,optimistic_estimate=array_with_more_true(x,missing_size)
    #print(optimistic_estimate)
    return optimistic_estimate


In [2058]:
assert goal_check(State(set(range(NUM_SETS)), set())), "Probelm not solvable"


In [2059]:
#we need a queue that is the frontier
frontier=PriorityQueue()
#define the initial state
frontier.put(State(set(),set(range(NUM_SETS))))
current_state=frontier.get()
contatore=0
while not goal_check(current_state):
    for action in current_state[1]:
     contatore+=1
     new_state=State (current_state.taken ^ {action}, current_state.not_taken ^{action})
     print(new_state)

     frontier.put((((h(new_state)-len(new_state.taken))),new_state))
     #print("mancano:",PROBLEM_SIZE-sum(covered(new_state)))
     #print("h:",h(new_state), "costo attuale:",len(new_state))
    _,current_state=frontier.get()
print(contatore)

State(taken={0}, not_taken={1, 2, 3, 4, 5, 6, 7, 8, 9})
mancano: 3 e possiamo usare nel caso migliore 1 sets e sono:
 [array([ True, False,  True,  True, False])] 


State(taken={1}, not_taken={0, 2, 3, 4, 5, 6, 7, 8, 9})
mancano: 3 e possiamo usare nel caso migliore 1 sets e sono:
 [array([ True, False,  True,  True, False])] 


State(taken={2}, not_taken={0, 1, 3, 4, 5, 6, 7, 8, 9})
mancano: 4 e possiamo usare nel caso migliore 2 sets e sono:
 [array([ True, False,  True,  True, False]), array([False,  True,  True, False,  True])] 


State(taken={3}, not_taken={0, 1, 2, 4, 5, 6, 7, 8, 9})
mancano: 5 e possiamo usare nel caso migliore 2 sets e sono:
 [array([ True, False,  True,  True, False]), array([False,  True,  True, False,  True])] 


State(taken={4}, not_taken={0, 1, 2, 3, 5, 6, 7, 8, 9})
mancano: 2 e possiamo usare nel caso migliore 1 sets e sono:
 [array([False,  True,  True, False,  True])] 


State(taken={5}, not_taken={0, 1, 2, 3, 4, 6, 7, 8, 9})
mancano: 2 e possiamo usar

# Solution with Dijkstra

In [2060]:
frontier_=PriorityQueue()
#define the initial state
frontier_.put(State(set(),set(range(NUM_SETS))))
current_state_=frontier_.get()
contatore_=0
while not goal_check(current_state_):
    for action in current_state_[1]:
     contatore_+=1
     new_state=State (current_state_.taken ^ {action}, current_state_.not_taken ^{action})
     frontier_.put((-len(current_state_.taken),new_state))
    _,current_state_=frontier_.get()

#  Prof Solution 
I get more or less the same results, never better but often equal

In [2061]:
def h3(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken


frontier_3=PriorityQueue()
#define the initial state
frontier_3.put(State(set(),set(range(NUM_SETS))))
current_state_3=frontier_3.get()
contatore_3=0
while not goal_check(current_state_3):
    for action in current_state_3[1]:
     contatore_3+=1
     new_state_3=State (current_state_3.taken ^ {action}, current_state_3.not_taken ^{action})
     #print(new_state_3)

     frontier_3.put((((h3(new_state_3)-len(new_state_3.taken))),new_state_3))
     #print("mancano:",PROBLEM_SIZE-sum(covered(new_state)))
     #print("h:",h(new_state), "costo attuale:",len(new_state))
    _,current_state_3=frontier_3.get()


In [2062]:
print("My method use ",contatore, "steps and give us: ",current_state)
print("Method with Dijkstra use",contatore_,"steps and give us: ", current_state_)
print("Method Prof use",contatore_3,"steps and give us: ", current_state_3)


My method use  27 steps and give us:  State(taken={0, 1, 5}, not_taken={2, 3, 4, 6, 7, 8, 9})
Method with Dijkstra use 45 steps and give us:  State(taken={0, 1, 2, 3, 4, 5}, not_taken={6, 7, 8, 9})
Method Prof use 19 steps and give us:  State(taken={4, 5}, not_taken={0, 1, 2, 3, 6, 7, 8, 9})
